In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram
import matplotlib.pyplot as plt


spotify = pd.read_csv('genres_v2.csv')
spotify = spotify.drop(columns=['analysis_url', 'track_href', 'uri', 'id','title'])
spotify.head()



/var/folders/h4/hynycmgd3x59kzk4xdgvp4w00000gn/T/ipykernel_18762/2050498044.py:9: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  spotify = pd.read_csv('genres_v2.csv')


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,duration_ms,time_signature,genre,song_name,Unnamed: 0
0,0.831,0.814,2,-7.364,1,0.4200,0.0598,0.013400,0.0556,0.3890,156.985,audio_features,124539,4,Dark Trap,Mercury: Retrograde,NaN
1,0.719,0.493,8,-7.230,1,0.0794,0.4010,0.000000,0.1180,0.1240,115.080,audio_features,224427,4,Dark Trap,Pathology,NaN
2,0.850,0.893,5,-4.783,1,0.0623,0.0138,0.000004,0.3720,0.0391,218.050,audio_features,98821,4,Dark Trap,Symbiote,NaN
3,0.476,0.781,0,-4.710,1,0.1030,0.0237,0.000000,0.1140,0.1750,186.948,audio_features,123661,3,Dark Trap,ProductOfDrugs (Prod. The Virus and Antidote),NaN
4,0.798,0.624,2,-7.668,1,0.2930,0.2170,0.000000,0.1660,0.5910,147.988,audio_features,123298,4,Dark Trap,Venom,NaN


In [47]:


from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

spotify.fillna(spotify.mean(), inplace=True)
# Standardizing the features
spotify_scaled = StandardScaler().fit_transform(spotify.drop(columns=['song_name', 'type', 'Unnamed: 0', 'time_signature', 'duration_ms', 'mode', 'genre'] , axis=1))

# Performing PCA
pca = PCA(n_components=0.95)  # retain 95% of the variance
principalComponents = pca.fit_transform(spotify_scaled)

# Creating a DataFrame with principal components
spotify_pca = pd.DataFrame(data=principalComponents)

agg_cluster_model = AgglomerativeClustering(n_clusters=15)
agg_clusters = agg_cluster_model.fit_predict(spotify_pca)


: 

In [30]:
spotify['cluster'] = agg_clusters

In [40]:
def recommend_songs(song_name, data, pca_data, features, n_recommendations=5):
    if song_name not in data['song_name'].values:
        print("Song not found in the dataset.")
        return None
    
    # Finding the cluster of the input song
    song_cluster = data.loc[data['song_name'] == song_name, 'cluster'].iloc[0]

    # Filter songs from the same cluster
    cluster_songs = data[data['cluster'] == song_cluster]
    
    # Get the PCA components of the input song
    song_index = data[data['song_name'] == song_name].index[0]
    song_pca_features = pca_data[song_index]

    # Calculate the similarity (e.g., using Euclidean distance) between the input song and other songs in the cluster
    cluster_songs['similarity'] = cluster_songs.apply(lambda x: np.linalg.norm(pca_data[x.name] - song_pca_features), axis=1)
    
    # Sort the songs based on similarity scores
    recommended_songs = cluster_songs.sort_values('similarity')[['song_name', 'similarity']]

    return recommended_songs.head(n_recommendations)

In [ ]:
# Define the features
features = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence']

# Example: Recommend songs similar to a given song name
song_to_recommend = 'Song Name Here'  # Replace with a song name from your dataset

# Call the recommendation function
recommendations = recommend_songs(song_to_recommend, spotify, features)

# Display the recommendations
if recommendations is not None:
    print("Recommendations for the song:", song_to_recommend)
    print(recommendations)
else:
    print("No recommendations available.")